<a href="https://colab.research.google.com/github/DataMonarch/fashion_mnist/blob/main/fashion_mnist_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
import numpy as np
import os
import distutils
import datetime

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [7]:
x_train.shape

(60000, 28, 28, 1)

(60000,)

In [10]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

In [12]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(BatchNormalization(input_shape = x_train.shape[1:]))
    model.add(Conv2D(64, (5, 5), padding='same', activation='elu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.05))

    model = tf.keras.models.Sequential()
    model.add(BatchNormalization(input_shape = x_train.shape[1:]))
    model.add(Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.05))

    model = tf.keras.models.Sequential()
    model.add(BatchNormalization(input_shape = x_train.shape[1:]))
    model.add(Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.05))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('elu'))
    model.add(Dropout(0.05))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    return model



In [21]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [22]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [25]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
  
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

model.save_weights('./fashion_mnist.h5', overwrite=True)

Epoch 1/17
60/60 [==============================] - 5s 21ms/step - loss: 0.5958 - sparse_categorical_accuracy: 0.7985
Epoch 2/17
60/60 [==============================] - 1s 21ms/step - loss: 0.3462 - sparse_categorical_accuracy: 0.8771
Epoch 3/17
60/60 [==============================] - 1s 21ms/step - loss: 0.2980 - sparse_categorical_accuracy: 0.8939
Epoch 4/17
60/60 [==============================] - 1s 21ms/step - loss: 0.2657 - sparse_categorical_accuracy: 0.9048
Epoch 5/17
60/60 [==============================] - 1s 21ms/step - loss: 0.2393 - sparse_categorical_accuracy: 0.9139
Epoch 6/17
60/60 [==============================] - 1s 21ms/step - loss: 0.2130 - sparse_categorical_accuracy: 0.9230
Epoch 7/17
60/60 [==============================] - 1s 21ms/step - loss: 0.1946 - sparse_categorical_accuracy: 0.9291
Epoch 8/17
60/60 [==============================] - 1s 21ms/step - loss: 0.1745 - sparse_categorical_accuracy: 0.9360
Epoch 9/17
60/60 [==============================] - 1s 2